# Library

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import minmax_scale, RobustScaler,QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, plot_confusion_matrix, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import train_test_split
from IPython.display import HTML, display
import xgboost as xgb
import warnings
from sklearn.ensemble import RandomForestClassifier

# Dataset

In [31]:
df = pd.read_csv("../Models/data_tofit.csv")
df

,Age,Gender,Education,Nscore,Escore,Oscore,Ascore,Cscore,SS,Amphet,Amyl,Benzos,Cannabis,Heroin,Ketamine,LSD,Meth,Nicotine,VSA,Addiction
0,0.49788,0.48246,-0.05921,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-1.18084,1,0,1,0,0,0,0,0,1,0,Low Addiction
1,-0.07854,-0.48246,1.98437,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.21575,1,1,0,1,0,1,1,1,1,0,Addiction without psychologic effect
2,0.49788,-0.48246,-0.05921,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,0.40148,0,0,0,1,0,0,0,0,0,0,Low Addiction
3,-0.95197,0.48246,1.16365,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.18084,0,0,1,1,0,1,0,0,1,0,Low Addiction
4,0.49788,0.48246,1.98437,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21575,0,0,0,1,0,0,0,0,1,0,Low Addiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,-0.95197,0.48246,-0.61113,-1.19430,1.74091,1.88511,0.76096,-1.13788,1.92173,0,0,0,1,0,0,1,0,0,1,Addiction without psychologic effect
1881,-0.95197,-0.48246,-0.61113,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.76540,0,0,0,1,0,0,1,1,1,0,Addiction without psychologic effect
1882,-0.07854,0.48246,0.45468,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,-0.52593,1,1,1,1,0,1,1,0,1,0,Addiction with psychologic effect
1883,-0.95197,0.48246,-0.61113,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.22470,0,0,0,1,0,0,1,0,1,0,Addiction with psychologic effect


# Model

In [32]:
# Split train test

X_train, X_test, y_train, y_test = train_test_split(df.drop("Addiction", axis = 1), df["Addiction"], test_size=0.35, random_state = 42,\
                                                   stratify = df["Addiction"])

In [33]:
# train dataset

train_dataset = pd.concat([X_train, y_train],axis=1)
train_dataset.head()

,Age,Gender,Education,Nscore,Escore,Oscore,Ascore,Cscore,SS,Amphet,Amyl,Benzos,Cannabis,Heroin,Ketamine,LSD,Meth,Nicotine,VSA,Addiction
740,0.49788,0.48246,0.45468,0.62967,0.16767,-0.71727,-0.91699,-0.89891,-1.18084,0,0,1,0,0,0,0,0,1,0,Low Addiction
619,-0.95197,-0.48246,-1.22751,0.52135,-1.37639,-0.45174,1.45039,-1.78169,0.40148,0,0,0,1,0,1,0,0,1,1,Addiction with psychologic effect
1254,-0.07854,0.48246,1.16365,1.49158,-1.23177,-0.71727,-0.91699,-0.00665,-0.52593,0,0,0,0,0,0,0,0,0,0,Low Addiction
1185,-0.07854,0.48246,0.45468,1.23461,-0.30033,0.88309,0.43852,-0.00665,0.07987,0,1,0,1,0,0,0,0,1,0,Addiction with psychologic effect
1216,-0.07854,0.48246,1.16365,0.31287,1.28610,-0.17779,-0.76096,-1.01450,-0.52593,1,0,1,1,0,1,1,1,1,0,Addiction without psychologic effect


In [34]:
train_dataset.shape

(1225, 20)

In [35]:
# hyperparameters

parameters = {'bootstrap': [True, False],\
 'max_depth': [2, 3, 4, 5, 6, 7, 8],\
 'max_features': ['auto', 'sqrt'],\
 'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9],\
 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],\
 'n_estimators': [50, 100, 150, 200, 250, 300]
 }

In [38]:
grid = GridSearchCV(RandomForestClassifier, parameters, cv = 5)
grid.fit(X_train, y_train)

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [ ]:
print(grid.best_score_, grid.best_estimator_)